##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. To run all the code in the notebook, select **Runtime** > **Run all**. To run the code cells one at a time, hover over each cell and select the **Run cell** icon.

![Run cell icon](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/images/beginner/run_cell_icon.png?raw=1)

Install Object detection API an YAML

In [1]:
!pip install tensorflow-object-detection-api
!pip install tensorflow pillow pyyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.0/739.0 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.3 MB/s eta 0:00:00
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844489 sha256=86139a37b66baf36b149aa7843bca0

Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Set up TensorFlow

Import TensorFlow into your program to get started:

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import xml.etree.ElementTree as ET
import numpy as np
import yaml

then run the rest of the code

In [5]:
# Define your model architecture
def create_model(input_shape, num_classes):
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Load class names from label_map
def load_class_names(label_map):
    with open(label_map, 'r') as file:
        data = yaml.safe_load(file)
        class_names = data['names']
    return class_names

# Parse annotation file
def parse_annotation(annotation_path, class_names, num_classes):
    try:
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        label = np.zeros(num_classes)
        for obj in root.findall('object'):
            class_name = obj.find('name').text
            class_id = class_names.index(class_name)
            label[class_id] = 1
        return label
    except FileNotFoundError:
        print(f"Annotation file not found: {annotation_path}. Skipping this file.")
        return None

# Data generator
def data_generator(image_files, images_dir, annotations_dir, class_names, batch_size, input_shape):
    num_classes = len(class_names)

    def generator():
        for image_file in image_files:
            image_path = os.path.join(images_dir, image_file)
            annotation_file = os.path.splitext(image_file)[0] + ".xml"
            annotation_path = os.path.join(annotations_dir, annotation_file)

            image = tf.keras.preprocessing.image.load_img(image_path, target_size=input_shape[:2])
            image = tf.keras.preprocessing.image.img_to_array(image)

            label = parse_annotation(annotation_path, class_names, num_classes)
            if label is not None:
                yield image, label

    dataset = tf.data.Dataset.from_generator(generator, output_signature=(
        tf.TensorSpec(shape=input_shape, dtype=tf.float32),
        tf.TensorSpec(shape=(num_classes,), dtype=tf.float32)
    ))
    dataset = dataset.batch(batch_size)
    return dataset

# Train your model
def train_model(data_yaml, batch_size=64, epochs=1):
    with open(data_yaml, 'r') as file:
        data = yaml.safe_load(file)

    class_names = data['names']
    num_classes = len(class_names)
    input_shape = (640, 640, 3)  # Update this to match your image dimensions

    train_images_dir = os.path.join('/content/drive/MyDrive/cards_training', data['train'])
    train_annotations_dir = os.path.join('/content/drive/MyDrive/cards_training/train/labels_voc')
    val_images_dir = os.path.join('/content/drive/MyDrive/cards_training', data['val'])
    val_annotations_dir = os.path.join('/content/drive/MyDrive/cards_training/valid/labels_voc')

    train_image_files = [f for f in os.listdir(train_images_dir) if f.endswith(".jpg") or f.endswith(".png")]
    val_image_files = [f for f in os.listdir(val_images_dir) if f.endswith(".jpg") or f.endswith(".png")]

    train_dataset = data_generator(train_image_files, train_images_dir, train_annotations_dir, class_names, batch_size, input_shape)
    val_dataset = data_generator(val_image_files, val_images_dir, val_annotations_dir, class_names, batch_size, input_shape)

    model = create_model(input_shape, num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Add progress printing during training
    class PrintProgress(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            logs = logs or {}
            print(f"Epoch {epoch + 1}/{self.params['epochs']}, Loss: {logs.get('loss', 0):.4f}, Accuracy: {logs.get('accuracy', 0):.4f}, Val Loss: {logs.get('val_loss', 0):.4f}, Val Accuracy: {logs.get('val_accuracy', 0):.4f}")

    model.fit(train_dataset, validation_data=val_dataset, epochs=epochs, callbacks=[PrintProgress()])

    # Save the trained model
    model.save('/content/drive/MyDrive/cards_training/saved_model/my_model.keras')

# Convert the trained model to TensorFlow Lite format
def convert_to_tflite():
    # Load the trained model
    model = tf.keras.models.load_model('/content/drive/MyDrive/cards_training/saved_model/my_model.keras')

    # Convert the model to TensorFlow Lite format
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # Save the TensorFlow Lite model
    with open('/content/drive/MyDrive/cards_training/saved_model/model.tflite', 'wb') as f:
        f.write(tflite_model)

if __name__ == '__main__':
    data_yaml = '/content/drive/MyDrive/cards_training/data.yaml'

    print("Training the model...")
    train_model(data_yaml)
    print("Model trained successfully.")

    print("Converting the model to TensorFlow Lite format...")
    convert_to_tflite()
    print("Model converted and saved successfully.")

Training the model...
Epoch 1/50
    104/Unknown 1496s 14s/step - accuracy: 0.0579 - loss: 91195.7969Annotation file not found: /content/drive/MyDrive/cards_training/train/labels_voc/150002502_jpg.rf.0daef79388bf6bda71149bb6c464027f (1).xml. Skipping this file.
    181/Unknown 2608s 14s/step - accuracy: 0.0532 - loss: 431705504.0000

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


Epoch 1/50, Loss: 5363449344.0000, Accuracy: 0.0396, Val Loss: 162743533568.0000, Val Accuracy: 0.0000
181/181 ━━━━━━━━━━━━━━━━━━━━ 2624s 14s/step - accuracy: 0.0531 - loss: 458803008.0000 - val_accuracy: 0.0000e+00 - val_loss: 162743533568.0000
Epoch 2/50
103/181 ━━━━━━━━━━━━━━━━━━━━ 40s 519ms/step - accuracy: 0.0214 - loss: 290581020672.0000Annotation file not found: /content/drive/MyDrive/cards_training/train/labels_voc/150002502_jpg.rf.0daef79388bf6bda71149bb6c464027f (1).xml. Skipping this file.
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 514ms/step - accuracy: 0.0198 - loss: 778874781696.0000Epoch 2/50, Loss: 2345488875520.0000, Accuracy: 0.0176, Val Loss: 21350672498688.0000, Val Accuracy: 0.0000
181/181 ━━━━━━━━━━━━━━━━━━━━ 94s 515ms/step - accuracy: 0.0198 - loss: 787482542080.0000 - val_accuracy: 0.0000e+00 - val_loss: 21350672498688.0000
Epoch 3/50
103/181 ━━━━━━━━━━━━━━━━━━━━ 41s 530ms/step - accuracy: 0.0213 - loss: 11846038847488.0000Annotation file not found: /content/drive/MyDrive/

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=/content/drive/MyDrive/cards_training/saved_model/my_model.